# Pop atoms
Let's find the musical atoms that make up the most popular hits of the last decades

## Import functions and libraries

In [4]:
from functions import *
from os.path import exists

## Get data

In [5]:
if not exists("df_ranks_weekly.csv"):
    # Get all the Billboard data.
    # Billboard collects the most popular songs weekly since 1958. This is the data we will use for this project.
    if exists("df_ranks_weekly_temp.csv"):
        temp_csv_path = "df_ranks_weekly_temp.csv"
    else:
        temp_csv_path = ""
        
    billboard_ranks_raw = create_weekly_ranks_df("1958-01-01", "2024-08-31", csv_path = temp_csv_path)
    
    # Save the cleaned data to a CSV file to avoid having to recalculate it.
    billboard_ranks_raw.to_csv("df_ranks_weekly.csv", index=False)


In [ ]:
if not exists("df_ranks_peaks.csv"):    
# Get peak values. 
    # Some songs are listed more than once, as they come up several times in the charts for different weeks. 
    # We will only keep one entry for each song, calculate the peak position, maximum number of weeks on the chart, and date that peak position was reached for each song.  

#    billboard_ranks_raw = pd.read_csv("billboard_rank_raw.csv")
    billboard_ranks_raw = pd.read_csv("df_ranks_weekly.csv")

    billboard_ranks_peak = calculate_rank_peak_data(billboard_ranks_raw)

    # Save the cleaned data to a CSV file.
    billboard_ranks_peak.to_csv("df_ranks_peaks.csv.csv", index=False)

In [ ]:
## Add audio analysis data

if not exists("df_audio_analysis.csv"):

    # Load the cleaned data
    billboard_ranks_peak = pd.read_csv("billboard_rank_peaks.csv")

    # add audio analysis columns
    audio_analysis_df = create_audio_analysis_df (billboard_ranks_peak)

    # Save the cleaned data to a CSV file.
    audio_analysis_df.to_csv("df_audio_analysis.csv", index=False)


## Clean data

In [ ]:
# read data from csv
audio_analysis_df = pd.read_csv("audio_analysis.csv")

## Explore data